In [1]:
import subprocess
from pyspark.sql.types import IntegerType
from functools import reduce
from pyspark.sql import DataFrame

# w1=subprocess.Popen("hdfs dfs -ls hdfs://localhost:9000/user/akash/trafficCount/aggre| awk '{print $8}' | awk 'NR!=1' ",
# shell=True,
# stdout=subprocess.PIPE,
# stderr=subprocess.STDOUT)

# w2=subprocess.Popen("hdfs dfs -ls hdfs://localhost:9000/user/akash/trafficCount/lockdownweek| awk '{print $8}' | awk 'NR!=1' ",
# shell=True,
# stdout=subprocess.PIPE,
# stderr=subprocess.STDOUT)


In [2]:

def hdfs_file_read(file):
    rdd = spark.read.csv(file,header=True)
    # rdd=rdd.drop("axlespacings", "axleweights", "numberofaxles","validitycode","duration","temperature","weight","hour","minute","second","millisecond")
    return rdd

In [3]:
def get_Dataframe(x):
    w1=subprocess.Popen(x,
    shell=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT)
    res = list()
    for line in w1.stdout.readlines():
        file = line.decode().strip()
        vehicle_data1= hdfs_file_read(file)
        vehicle_data1= vehicle_data1.withColumn("VehicleCount", vehicle_data1["VehicleCount"].cast(IntegerType()))
        vehicle_data1= vehicle_data1.withColumn("cosit", vehicle_data1["cosit"].cast(IntegerType()))
        week1_VehicleCount=vehicle_data1.groupBy(vehicle_data1.cosit).sum('VehicleCount').sort('sum(VehicleCount)',ascending=False)
        week1_VehicleCount=week1_VehicleCount.withColumnRenamed("sum(VehicleCount)", "w1TotalCount")
#         week1_VehicleCount.show()
    #     res.append(week1_VehicleCount)
        res.append(week1_VehicleCount)
    union1 = reduce(DataFrame.unionAll, res)
    union1=union1.groupBy(union1.cosit).sum('w1TotalCount').sort('sum(w1TotalCount)',ascending=False)
    union1=union1.withColumnRenamed("sum(w1TotalCount)", "TotalCount")
    return union1
# union1.filter(union1.cosit ==1502).show()

In [4]:
w1="hdfs dfs -ls hdfs://localhost:9000/user/akash/trafficCount/aggre| awk '{print $8}' | awk 'NR!=1' "
w2="hdfs dfs -ls hdfs://localhost:9000/user/akash/trafficCount/lockdownweek| awk '{print $8}' | awk 'NR!=1' "
L=[w1,w2]
for i in range(0,2):
        j=str(i)
        globals()[f"week{j}"] =  get_Dataframe(L[i])
        
# week0.show()
# week1.show()
week0.createOrReplaceTempView("week1")

week1.createOrReplaceTempView("week2")

finalReductionCount=spark.sql("SELECT w1.cosit,(w1.TotalCount - w2.TotalCount) Difference FROM week1 w1 INNER JOIN week2 w2 ON w1.cosit = w2.cosit ORDER BY Difference DESC LIMIT 10")
finalReductionCount.show()


+-----+----------+
|cosit|Difference|
+-----+----------+
| 1502|    789723|
| 1508|    786520|
| 1503|    784145|
| 1014|    759632|
| 1501|    749373|
| 1509|    729802|
| 1500|    707956|
| 1504|    669135|
| 1505|    625228|
| 1506|    605283|
+-----+----------+

